In [1]:
import pandas as pd


df = pd.read_csv("C:/Users/mmrha/Desktop/cmu/Independent study/python/abalonemin.csv")
df=df.sort_values(by=["sex"])
df=df.reset_index(drop=True)
df

,sex,length,diameter,height,wholeweight,shuckedweight,visceraweight,shellweight
0,F,110,91,37,2174,1173,688,791
1,F,74,57,26,972,303,261,366
2,F,66,60,33,1170,468,381,472
3,F,80,63,36,909,289,218,436
4,F,74,61,30,1123,450,278,428
5,F,105,82,43,2010,808,455,831
6,F,100,88,45,1741,715,537,696
7,F,116,101,37,2304,1283,637,797
8,F,86,77,36,1781,613,408,843
9,F,86,67,27,1106,569,356,463


In [2]:
df = pd.get_dummies(df)
print(df)

    length  diameter  height  wholeweight  shuckedweight  visceraweight  \
0      110        91      37         2174           1173            688   
1       74        57      26          972            303            261   
2       66        60      33         1170            468            381   
3       80        63      36          909            289            218   
4       74        61      30         1123            450            278   
5      105        82      43         2010            808            455   
6      100        88      45         1741            715            537   
7      116       101      37         2304           1283            637   
8       86        77      36         1781            613            408   
9       86        67      27         1106            569            356   
10     112        94      40         2172            994            781   
11     133       110      39         2423           1458            878   
12      93        74     

In [3]:
plt.plot(df["sex_F"])
plt.plot(df["sex_I"])
plt.plot(df["sex_M"])


NameError: name 'plt' is not defined

In [ ]:

from keras.layers import Input
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras import backend as K


latent_dimension = 1
batch_size = 20
hidden_nodes = 16


input_encoder = Input(shape=(10,), name="Input_Encoder")
batch_normalize1 = BatchNormalization()(input_encoder)
hidden_layer = Dense(hidden_nodes, activation="relu", name="Hidden_Encoding")(
    batch_normalize1
)
batch_normalize2 = BatchNormalization()(hidden_layer)
z = Dense(latent_dimension, name="Mean")(batch_normalize2)

In [ ]:
from keras import Model


encoder = Model(input_encoder, z)

In [ ]:
input_decoder = Input(shape=(latent_dimension,), name="Input_Decoder")
batch_normalize1 = BatchNormalization()(input_decoder)
decoder_hidden_layer = Dense(hidden_nodes, activation="relu", name="Hidden_Decoding")(
    batch_normalize1
)
batch_normalize2 = BatchNormalization()(decoder_hidden_layer)
decoded = Dense(10, activation="linear", name="Decoded")(batch_normalize2)

In [ ]:
decoder = Model(input_decoder, decoded, name="Decoder")

In [ ]:
encoder_decoder = decoder(encoder(input_encoder))

ae = Model(input_encoder, encoder_decoder)
ae.summary()

In [ ]:
from tensorflow.random import set_seed


set_seed(2021)
ae.compile(loss="mean_squared_error", optimizer="adam")
history = ae.fit(
    df, df, shuffle=True, epochs=7000, batch_size=20, validation_split=0.2, verbose=0
).history

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


sns.set(font_scale=2)
sns.set_style("white")


def model_analysis(history):
    train_loss = history["loss"]
    val_loss = history["val_loss"]
    t = np.linspace(1, len(train_loss), len(train_loss))

    plt.figure(figsize=(16, 12))
    plt.title("Mean squared error")
    sns.lineplot(x=t, y=train_loss, label="Train", linewidth=3)
    sns.lineplot(x=t, y=val_loss, label="Validation", linewidth=3)
    plt.xlabel("Epochs")

    plt.legend()
    plt.savefig("FirstNet.png", dpi=400)
    plt.show()
    print(f"Training MSE = {np.sqrt(train_loss[-1])}")
    print(f"Validation MSE = {np.sqrt(val_loss[-1])}")


model_analysis(history)

In [ ]:
origin = pd.DataFrame(ae.predict(df))
origin.columns = df.columns
origin

In [ ]:
plt.plot(origin["sex_F"])
plt.plot(origin["sex_I"])
plt.plot(origin["sex_M"])

In [ ]:
ae.predict(df)[0,:]

df.iloc[0, :]

In [ ]:
from statsmodels.distributions.empirical_distribution import ECDF


ecdf = ECDF(encoder.predict(df)[:, 0])
plt.figure(figsize=(16, 12))
plt.title("Empirical distribution function z")
x = (-9, -8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13)
plt.yticks(np.linspace(0, 1, 11))
plt.xticks(x)
plt.grid()
plt.plot(x, ecdf(x), linewidth=3)
plt.savefig("EmpiricalDF.png", dpi=400)

In [ ]:
from scipy.interpolate import interp1d


x = (-9, -8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13)
sample_edf_values_at_slope_changes = [ecdf(i) for i in x]
inverted_edf = interp1d(sample_edf_values_at_slope_changes, x)

In [ ]:
from numpy.random import uniform
from numpy.random import seed
N = 10000


output = pd.DataFrame(decoder(inverted_edf(uniform(0.02, 0.95, N))).numpy())
output.columns = df.columns

print(output)

In [ ]:
plt.plot(output["sex_F"])
plt.plot(output["sex_I"])
plt.plot(output["sex_M"])

In [ ]:
output.iloc[0, :]

In [ ]:
output.to_csv("C:/Users/mmrha/Desktop/cmu/Independent study/python/abaloneauto5000_10000.csv")